In [ ]:
#!/usr/bin/env python3
import os
import time
import tempfile
import mimetypes
import subprocess
import gradio as gr
from google import genai
from google.genai import types

# -----------------------------
# Config & Globals
# -----------------------------
API_KEY = ""
CLIENT = None
CURRENT_STORE = None
FALLBACK_FILE = "/mnt/data/364d0f88-e8a5-4c4a-8eb2-e71fba0aadc3.png"

# -----------------------------
# Helpers
# -----------------------------
def _ensure_client(key):
    global CLIENT
    if CLIENT:
        return CLIENT
    CLIENT = genai.Client(api_key=key)
    return CLIENT

def _save_temp(path):
    fd, tmp = tempfile.mkstemp(suffix=os.path.splitext(path)[1])
    os.close(fd)
    with open(path, "rb") as f_in, open(tmp, "wb") as f_out:
        f_out.write(f_in.read())
    return tmp

# -----------------------------
# File Search Flow (from Google doc)
# -----------------------------
def process_file(file_obj):
    global CLIENT, CURRENT_STORE

    if CLIENT is None:
        return "⚠️ Please connect API first."

    src = file_obj.name if (file_obj and file_obj.name) else FALLBACK_FILE
    if not os.path.exists(src):
        return "❌ File not found."

    tmp = _save_temp(src)
    try:
        # 1) Create a File Search store
        store = CLIENT.file_search_stores.create(config={"display_name": f"store_{int(time.time())}"})
        CURRENT_STORE = store

        # 2) Upload + import file into that store
        op = CLIENT.file_search_stores.upload_to_file_search_store(
            file=tmp,
            file_search_store_name=store.name,
            config={"display_name": os.path.basename(src)}
        )

        # 3) Poll until import is done
        while not op.done:
            time.sleep(5)
            op = CLIENT.operations.get(op)

        return f"✅ File imported into File Search store: {store.name}"

    except Exception as e:
        return f"❌ Error: {e}"

    finally:
        if os.path.exists(tmp):
            os.remove(tmp)

def chat_logic(message, history):
    global CLIENT, CURRENT_STORE

    if CLIENT is None:
        return "⚠️ Connect API first."
    if CURRENT_STORE is None:
        return "⚠️ Upload a document first."

    # Use the FileSearch tool
    tool = types.Tool(
        file_search=types.FileSearch(
            file_search_store_names=[CURRENT_STORE.name]
        )
    )

    # Build the generate_content config
    config = types.GenerateContentConfig(
        tools=[tool],
        temperature=0.0,
        max_output_tokens=800
    )

    response = CLIENT.models.generate_content(
        model="gemini-2.5-flash",
        contents=[message],
        config=config
    )

    # The docs show response.text
    return response.text

# -----------------------------
# Gradio UI
# -----------------------------
def setup_client(api_key_input):
    key = (api_key_input or "").strip() or API_KEY
    if not key:
        return "⚠️ Please enter your Gemini API Key"
    try:
        _ensure_client(key)
        return "✅ Gemini client connected."
    except Exception as e:
        return f"❌ Error connecting: {e}"

with gr.Blocks() as app:
    gr.Markdown("## Gemini File Search Assistant (using official API)")

    with gr.Row():
        with gr.Column(scale=1):
            api_input = gr.Textbox(label="Gemini API Key", type="password")
            btn = gr.Button("Connect")
            status = gr.Markdown("Not connected")

            file_input = gr.File(label="Upload PDF / Image / Text")
            upload_status = gr.Markdown("")

        with gr.Column(scale=2):
            chat = gr.ChatInterface(fn=chat_logic, type="messages", title="Ask about your document")

    btn.click(setup_client, api_input, status)
    file_input.change(process_file, file_input, upload_status)

if __name__ == "__main__":
    app.launch()
